In [41]:
import pandas as pd
pd.set_option("display.max_columns", None)

In [42]:
units = pd.read_csv("data/clean_crmls_units.csv")
p_buildings = pd.read_csv("data/clean_crmls_potential_unit_buildings.csv")

/var/folders/_3/v349phgn4ljd3jkf3ll9p2cm0000gp/T/ipykernel_8570/1222423188.py:1: DtypeWarning: Columns (4,21) have mixed types. Specify dtype option on import or set low_memory=False.
  units = pd.read_csv("data/clean_crmls_units.csv")


In [43]:
print(f'units shape: {units.shape}, p_buildings shape: {p_buildings.shape}')
print(f'num buildings in units: {units["ParcelFullAddress"].nunique()}')
print(f'num buildings in p_buildings: {p_buildings["ParcelFullAddress"].nunique()}')
print(f'overlap: {units[units["ParcelFullAddress"].isin(p_buildings["ParcelFullAddress"])]["ParcelFullAddress"].nunique()}')
print(f'no overlap: {units[~units["ParcelFullAddress"].isin(p_buildings["ParcelFullAddress"])]["ParcelFullAddress"].nunique()}')

units shape: (1073990, 36), p_buildings shape: (45584, 36)
num buildings in units: 170659
num buildings in p_buildings: 15018
overlap: 15018
no overlap: 155641


## Identifying Building Complex By Unit Number

In [44]:
units_agg = units.groupby('ParcelFullAddress')['ogr_fid'].nunique().reset_index()
units_agg.columns = ['ParcelFullAddress', 'unit_count']

In [45]:
building_complex = p_buildings.merge(right=units_agg, right_on=['ParcelFullAddress', 'unit_count'], left_on=['ParcelFullAddress', 'NumberofUnits'])

In [46]:
building_complex.shape

(4324, 37)

In [47]:
building_complex['ParcelFullAddress'].nunique()

1262

In [48]:
units[~units['ParcelFullAddress'].isin(building_complex['ParcelFullAddress'])]['ParcelFullAddress'].nunique()

169397

In [49]:
building_complex.groupby('ParcelFullAddress')['ogr_fid'].nunique().sort_values(ascending=False)[building_complex.groupby('ParcelFullAddress')['ogr_fid'].nunique().sort_values(ascending=False) > 1]

ParcelFullAddress
6400  SURFSIDE LN CARLSBAD CA 92011               1622
710  POWELL ST SAN FRANCISCO CA 94108              809
28891    VALLEY CENTER CA 92082                    394
1717  NOVATO BLVD NOVATO CA 94947                   29
2535 W WINTON AVE HAYWARD CA 94545                  15
                                                  ... 
3016  FILBERT ST OAKLAND CA 94608                    2
1022  POWELL ST SAN FRANCISCO CA 94108               2
1901  PRESCOTT RD MODESTO CA 95350                   2
13452  WASHINGTON BLVD MARINA DEL REY CA 90292       2
9957  MEDFORD AVE OAKLAND CA 94603                   2
Name: ogr_fid, Length: 74, dtype: int64

In [50]:
p_buildings[p_buildings['ParcelFullAddress'].isin(building_complex['ParcelFullAddress'])]

,ogr_fid,ParcelFullAddress,stunitprfx,stunitnum,parcelapn,taxapn,fips,parcelid,Upi,Assessments,AssesseeOwnerName,AssesseeMail_CityName,AssesseeMail_StateCode,TotalAssessedValue,AssessmentYear,TaxAmount,TaxYear,BuildingArea,MainBuildingAreaIndicator,YearBuilt,NumberOfBuildings,NumberOfStories,TotalNumberofRooms,NumberofUnits,NumberofBedrooms,NumberofBaths,LatestValidPrice,AssessedLandValue,AssessedImprovementValue,TaxRateCodeArea,Owner1FirstName,Owner1LastName,Owner2Firstname,Owner2MiddleName,Owner2LastName,ResoUPI
29,29247,2741 GARBER ST BERKELEY CA 94705,NaN,NaN,54-1707-21,54-1707-21,6001,100660192_173170781,US-06001-N-54170721-R-N,[{'AssesseeOwnerName': 'NOT AVAILABLE FROM THE...,NOT AVAILABLE FROM THE COUNTY,BERKELEY,CA,0,2021,NaN,2021,7778,L,1924.0,1,3,28,6,9,600,0,0,0,13-000,NaN,NOT AVAILABLE FROM THE COUNTY,NaN,NaN,NaN,US-06001-N-54170721-R-N
38,2122984,2117 E AROMA DR WEST COVINA CA 91791,NaN,NaN,8493041072,8493-041-072,6037,510684071_562929,US-06037-N-8493041072-R-N,"[{'AssesseeOwnerName': 'SUAZO ARCADIA', 'Asses...",SUAZO ARCADIA,WEST COVINA,CA,201087,2021,260460.0,2021,1231,L,1973.0,0,NaN,0,1,3,200,0,77107,123980,9-490,ARCADIA,SUAZO,NaN,NaN,NaN,US-06037-N-8493041072-R-N
82,84478,12 W RIANDA RD WATSONVILLE CA 95076,NaN,NaN,050-101-10,050-101-10,6087,100660192_16313877,US-06087-N-05010110-R-N,[{'AssesseeOwnerName': 'KNORNSCHILD ASENATH M ...,KNORNSCHILD ASENATH M (TRUSTEE),WATSONVILLE,CA,69551,2021,103362.0,2021,1266,L,1963.0,0,NaN,5,1,2,200,0,14665,54886,69-258,ASENATH M,KNORNSCHILD,NaN,NaN,ASENATH M KNORNSCHILD 2007 REVOCABLE TR,US-06087-N-05010110-R-N
114,825396,6672 BELL BLUFF AVE SAN DIEGO CA 92119,NaN,NaN,4562800532,456-280-05-32,6073,100660192_187487034,US-06073-N-4562800532-R-N,[{'AssesseeOwnerName': 'WILTON RONALD S & WEND...,WILTON RONALD S & WENDY L TRUST,SANTEE,CA,180953,2021,226004.0,2021,806,L,1972.0,0,NaN,0,1,2,100,130000,69595,111358,8-001,RON S,WILTON,WENDY L,NaN,WILTON,US-06073-N-4562800532-R-N
254,684195,909 ALEJO VISTA PALM SPRINGS CA 92262,NaN,NaN,508038013,508-038-013,6065,100660192_185538775,US-06065-N-508038013-R-N,"[{'AssesseeOwnerName': 'GRIFFIN PATRICK J', 'A...",GRIFFIN PATRICK J,SAN JUAN CAPISTRANO,CA,680068,2021,902166.0,2021,2258,L,2004.0,0,2,0,1,2,250,612500,169875,510193,11-052,PATRICK J,GRIFFIN,GREGORY B,NaN,BACCA,US-06065-N-508038013-R-N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45522,879966,309 ORLANDO ST EL CAJON CA 92021,NaN,NaN,4891611500,489-161-15-00,6073,100660192_187540552,US-06073-N-4891611500-R-N,"[{'AssesseeOwnerName': 'IVY ESTATES L L C', 'A...",IVY ESTATES L L C,SAN DIEGO,CA,1518662,2021,3309226.0,2021,0,NaN,NaN,0,NaN,0,37,0,0,650000,1304585,214077,3-003,NaN,IVY ESTATES L L C,NaN,NaN,NaN,US-06073-N-4891611500-R-N
45525,113012,1 BALDWIN AVE SAN MATEO CA 94401,NaN,NaN,109610010,109-610-010,6081,100660192_163263841,US-06081-N-109610010-R-N,[{'AssesseeOwnerName': 'BASS REAL ESTATE HOLDI...,BASS REAL ESTATE HOLDINGS LLC,MEDFORD,OR,4991946,2021,140384.0,2021,0,NaN,NaN,0,NaN,0,207,0,0,30500000,970861,4021085,12-023,NaN,BASS REAL ESTATE HOLDINGS LLC,NaN,NaN,NaN,US-06081-N-109610010-R-N
45566,820034,1165 HOLLISTER ST SAN DIEGO CA 92154,NaN,NaN,6340200300,634-020-03-00,6073,100660192_187481777,US-06073-N-6340200300-R-N,[{'AssesseeOwnerName': 'GOLDEN SANDS MOBILE ES...,GOLDEN SANDS MOBILE ESTATES L L C,SAN DIEGO,CA,1082217,2021,1268484.0,2021,0,NaN,NaN,0,NaN,0,55,0,0,0,811671,270546,8-035,NaN,GOLDEN SANDS MOBILE ESTATES L L C,NaN,NaN,NaN,US-06073-N-6340200300-R-N
45577,892643,1201 PARKER DAM RD EARP CA 92242,NaN,NaN,066113104,0661-131-04-W001,6071,533025219_12,US-06071-N-066113104W001-R-N,[{'AssesseeOwnerName': 'SUNDANCE RV RESORT LLC...,SUNDANCE RV RESORT LLC-#CACA-54885,PARKER,AZ,1201406,2021,1502593.0,2021,2000,T,1963.0,3,1,0,1,0,0,0,131873,1069533,96-001,NaN,SUNDANCE RV RESORT LLC-#CACA-54885,NaN,NaN,NaN,US-06071-N-066113104W001-R-N


In [51]:
p_buildings.groupby('ParcelFullAddress')['ogr_fid'].nunique().sort_values(ascending=False)[p_buildings.groupby('ParcelFullAddress')['ogr_fid'].nunique().sort_values(ascending=False) > 1]

ParcelFullAddress
8858  LAWRENCE WELK DR ESCONDIDO CA 92026    7131
6400  SURFSIDE LN CARLSBAD CA 92011          1676
1415  ORANGE AVE CORONADO CA 92118           1182
710  POWELL ST SAN FRANCISCO CA 94108         815
321  COAST BLVD LA JOLLA CA 92037             750
                                             ... 
12635  FRANKLIN CT CHINO CA 91710               2
12630  MONTE VISTA RD POWAY CA 92064            2
211 N CITRUS AVE ESCONDIDO CA 92027             2
1975  BARRYMORE CMN FREMONT CA 94538            2
3147 W OLIVE AVE FRESNO CA 93722                2
Name: ogr_fid, Length: 3248, dtype: int64

In [52]:
p_buildings.groupby(['ParcelFullAddress'])['parcelid'].nunique().sort_values(ascending=False)[p_buildings.groupby(['ParcelFullAddress'])['parcelid'].nunique().sort_values(ascending=False)>1]

ParcelFullAddress
8858  LAWRENCE WELK DR ESCONDIDO CA 92026           7131
6400  SURFSIDE LN CARLSBAD CA 92011                 1676
1415  ORANGE AVE CORONADO CA 92118                  1182
710  POWELL ST SAN FRANCISCO CA 94108                815
321  COAST BLVD LA JOLLA CA 92037                    750
                                                    ... 
370  VAQUEROS AVE RODEO CA 94572                       2
1011  PROFESSIONAL DR NAPA CA 94558                    2
1111 E TAHQUITZ CANYON WAY PALM SPRINGS CA 92262       2
2915 E 60TH PL HUNTINGTON PARK CA 90255                2
77  OVERPASS RD ROYAL OAKS CA 95076                    2
Name: parcelid, Length: 2436, dtype: int64

## Identifying Building Complex by NumberofUnits

In [53]:
max_unit = p_buildings.groupby('ParcelFullAddress')['NumberofUnits'].max().reset_index()
max_unit.columns = ['ParcelFullAddress', 'max_units']

In [54]:
total_units = units.groupby('ParcelFullAddress')['ogr_fid'].nunique().reset_index()
total_units

,ParcelFullAddress,ogr_fid
0,094 # 94 OFC HWY CAMPO CA 91906,1
1,1 KETTLEMAN CITY CA 93239,1
2,1 LAKESIDE CA 92040,1
3,1 1ST ST LAKEPORT CA 95453,3
4,1 3RD PL LONG BEACH CA 90802,42
...,...,...
170654,W11-B HARBOR DR SAN DIEGO CA 92101,1
170655,W14-F1 WESTERN ST SAN DIEGO CA 92101,1
170656,W4-C WESTERN ST SAN DIEGO CA 92027,1
170657,W7-A WESTERN ST SAN DIEGO CA 92101,1


In [55]:
f_buildings = p_buildings.merge(right=max_unit, right_on=['ParcelFullAddress', 'max_units'], left_on=['ParcelFullAddress', 'NumberofUnits'])

In [56]:
f_buildings.groupby('ParcelFullAddress')['ogr_fid'].nunique()[f_buildings.groupby('ParcelFullAddress')['ogr_fid'].nunique() > 1]

ParcelFullAddress
1  BED RED  HELENDALE CA 92342                          2
1  CHANDLER RANCH RD PALOS VERDES ESTATES CA 90274      4
1  CHRISTAMON  IRVINE CA 92620                          2
1  IRVINE PARK RD ORANGE CA 92869                       3
1  JOHNSON PIER  HALF MOON BAY CA 94019                74
                                                     ... 
SAN PABLO AVE EL CERRITO CA 94530                      16
SHELTON DR HOLLISTER CA 95023                           4
SHERMAN ISLAND EAST LEVE RD RIO VISTA CA 94571         52
SODA LAKE RD SANTA MARGARITA CA 93453                 593
SOLSTICE AVE CHULA VISTA CA 91915                       6
Name: ogr_fid, Length: 2326, dtype: int64

In [57]:
f_buildings[f_buildings['ParcelFullAddress'] == '1  BED RED  HELENDALE CA 92342']

,ogr_fid,ParcelFullAddress,stunitprfx,stunitnum,parcelapn,taxapn,fips,parcelid,Upi,Assessments,AssesseeOwnerName,AssesseeMail_CityName,AssesseeMail_StateCode,TotalAssessedValue,AssessmentYear,TaxAmount,TaxYear,BuildingArea,MainBuildingAreaIndicator,YearBuilt,NumberOfBuildings,NumberOfStories,TotalNumberofRooms,NumberofUnits,NumberofBedrooms,NumberofBaths,LatestValidPrice,AssessedLandValue,AssessedImprovementValue,TaxRateCodeArea,Owner1FirstName,Owner1LastName,Owner2Firstname,Owner2MiddleName,Owner2LastName,ResoUPI,max_units
34223,72780,1 BED RED HELENDALE CA 92342,NaN,NaN,046781136,0467-811-36-T016,6071,100660192_183356680,US-06071-N-046781136T016-R-N,[{'AssesseeOwnerName': 'CALLAHAN & ZALINSKY AS...,CALLAHAN & ZALINSKY ASSOCIATES LLC,BEAR,DE,1070,2021,3206.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,297,773,78-005,NaN,CALLAHAN & ZALINSKY ASSOCIATES LLC,NaN,NaN,NaN,US-06071-N-046781136T016-R-N,0
34224,73523,1 BED RED HELENDALE CA 92342,NaN,NaN,046781114,0467-811-14-T038,6071,100660192_183356652,US-06071-N-046781114T038-R-N,[{'AssesseeOwnerName': 'LAZARUS-HIBBARD JUDI E...,LAZARUS-HIBBARD JUDI ELLEN,SIMI VALLEY,CA,1073,2021,3212.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,298,775,78-005,JUDI,LAZARUS-HIBBARD,NaN,NaN,NaN,US-06071-N-046781114T038-R-N,0


In [58]:
units[units['ParcelFullAddress'] == '1  BED RED  HELENDALE CA 92342']

,ogr_fid,ParcelFullAddress,stunitprfx,stunitnum,parcelapn,taxapn,fips,parcelid,Upi,Assessments,AssesseeOwnerName,AssesseeMail_CityName,AssesseeMail_StateCode,TotalAssessedValue,AssessmentYear,TaxAmount,TaxYear,BuildingArea,MainBuildingAreaIndicator,YearBuilt,NumberOfBuildings,NumberOfStories,TotalNumberofRooms,NumberofUnits,NumberofBedrooms,NumberofBaths,LatestValidPrice,AssessedLandValue,AssessedImprovementValue,TaxRateCodeArea,Owner1FirstName,Owner1LastName,Owner2Firstname,Owner2MiddleName,Owner2LastName,ResoUPI
623336,72658,1 BED RED HELENDALE CA 92342,#,225,46781163,0467-811-63-T001,6071,100660192_183356677,US-06071-N-046781163T001-R-N,"[{'AssesseeOwnerName': 'MALENA JAMIE L', 'Asse...",MALENA JAMIE L,POMONA,CA,1237,2021,3406.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,308,929,78-005,JAMIE,MALENA,JODY,R,QUICK,US-06071-N-046781163T001-R-N
623337,72667,1 BED RED HELENDALE CA 92342,#,225,46781163,0467-811-63-T010,6071,100660192_183356677,US-06071-N-046781163T010-R-N,[{'AssesseeOwnerName': 'JORAWAR MAHENDRA & NIN...,JORAWAR MAHENDRA & NINETT,BRONX,NY,3096,2021,5681.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,929,2167,78-005,MAHENDRA,JORAWAR,NINETT,NaN,JORAWAR,US-06071-N-046781163T010-R-N
623338,72672,1 BED RED HELENDALE CA 92342,#,225,46781163,0467-811-63-T015,6071,100660192_183356677,US-06071-N-046781163T015-R-N,[{'AssesseeOwnerName': 'WALKER DEAN & KIMBERLY...,WALKER DEAN & KIMBERLY,BELLINGHAM,MA,1074,2021,3220.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,358,716,78-005,DEAN,WALKER,KIMBERLY,NaN,WALKER,US-06071-N-046781163T015-R-N
623339,72673,1 BED RED HELENDALE CA 92342,#,225,46781163,0467-811-63-T016,6071,100660192_183356677,US-06071-N-046781163T016-R-N,[{'AssesseeOwnerName': 'WALKER DEAN & KIMBERLY...,WALKER DEAN & KIMBERLY,BELLINGHAM,MA,1074,2021,3220.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,358,716,78-005,DEAN,WALKER,KIMBERLY,NaN,WALKER,US-06071-N-046781163T016-R-N
623340,72681,1 BED RED HELENDALE CA 92342,#,225,46781163,0467-811-63-T024,6071,100660192_183356677,US-06071-N-046781163T024-R-N,[{'AssesseeOwnerName': 'HAWKES GREGORY & CYNTH...,HAWKES GREGORY & CYNTHIA,EL CAJON,CA,1182,2021,3346.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,355,827,78-005,GREGORY,HAWKES,CYNTHIA,NaN,HAWKES,US-06071-N-046781163T024-R-N
623348,72765,1 BED RED HELENDALE CA 92342,#,126,46781136,0467-811-36-T001,6071,100660192_183356680,US-06071-N-046781136T001-R-N,[{'AssesseeOwnerName': 'STARKS GEORGE R & NADI...,STARKS GEORGE R & NADINE,ANAHEIM,CA,1074,2021,3220.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,358,716,78-005,GEORGE,STARKS,NADINE,NaN,STARKS,US-06071-N-046781136T001-R-N
623349,72768,1 BED RED HELENDALE CA 92342,#,126,46781136,0467-811-36-T004,6071,100660192_183356680,US-06071-N-046781136T004-R-N,[{'AssesseeOwnerName': 'WEAVER ROGER A & ASHLE...,WEAVER ROGER A & ASHLEY M,CARLSBAD,CA,1137,2021,3281.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,252,885,78-005,ROGER,WEAVER,ASHLEY,M,WEAVER,US-06071-N-046781136T004-R-N
623350,72773,1 BED RED HELENDALE CA 92342,#,126,46781136,0467-811-36-T009,6071,100660192_183356680,US-06071-N-046781136T009-R-N,[{'AssesseeOwnerName': 'FULLYLOVE NORMAN E & N...,FULLYLOVE NORMAN E & NICOLA,NaN,XX,1074,2021,3220.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,358,716,78-005,NORMAN,FULLYLOVE,NICOLA,NaN,FULLYLOVE,US-06071-N-046781136T009-R-N
623351,72785,1 BED RED HELENDALE CA 92342,#,126,46781136,0467-811-36-T021,6071,100660192_183356680,US-06071-N-046781136T021-R-N,"[{'AssesseeOwnerName': 'HIGUERA ERNESTINA', 'A...",HIGUERA ERNESTINA,LA PUENTE,CA,1137,2021,3281.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,252,885,78-005,ERNESTINA,HIGUERA,NaN,NaN,NaN,US-06071-N-046781136T021-R-N
623352,72786,1 BED RED HELENDALE CA 92342,#,126,46781136,0467-811-36-T022,6071,100660192_183356680,US-06071-N-046781136T022-R-N,"[{'AssesseeOwnerName': 'VORAGEN MARIANNE J', '...",VORAGEN MARIANNE J,WOODLAND HILLS,CA,903,2021,3005.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,258,645,78-005,MARIANNE,VORAGEN,NaN,NaN,NaN,US-06071-N-046781136T022-R-N
